Using Agentic Framework - Open AI SDK

In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [4]:
load_dotenv(override=True)

True

In [5]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
#We will define agents and add user system prompts to it - Three agents with different email styles

salesAgent1 = Agent(
    name = "Professional Agent",
    instructions = instructions1,
    model = "gpt-4o-mini"
)

salesAgent2 = Agent(
    name = "Professional Agent",
    instructions = instructions2,
    model = "gpt-4o-mini"
)


salesAgent3 = Agent(
    name = "Professional Agent",
    instructions = instructions3,
    model = "gpt-4o-mini"
)

In [ ]:
#We can wrap it all in a trace

from asyncio import gather


message = "Write a cold sales email"


with trace("Select the best email"):
    results = await asyncio.gather(
        Runner.run(salesAgent1, message),
        Runner.run(salesAgent2, message),
        Runner.run(salesAgent3, message)
    )
outputs = [result.final_output for result in results ]

for output in outputs:
    print(output + "\n\n")

In [10]:
#Now we define the sales picker to decide which is the best agent in this case

SalesPicker = Agent(
    name = "Sales Manager",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold sales email"

with trace("Best choice picker"):

    results = await asyncio.gather(
        Runner.run(salesAgent1, message),
        Runner.run(salesAgent2, message),
        Runner.run(salesAgent3, message)
    )
    
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(SalesPicker, emails)

    print(f"Best sales email:\n{best.final_output}")

A look at traces on open ai  shows how these agents have been wrapped by traces as seen below;

![My Image](traceshots.png)
